Manual config:

+ dtype for create and read function

+ shape to set_shape after reading

## Define serialize scheme

In [1]:
import tensorflow as tf
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def seri(image, mask):
  image = tf.io.serialize_tensor(image)
  mask = tf.io.serialize_tensor(mask)
  feature = {
      "image": _bytes_feature(image),
      "mask": _bytes_feature(mask)
  }
  return tf.train.Example(features = tf.train.Features(feature = feature)).SerializeToString()

## iterate and write the records

In [2]:
import tensorflow as tf
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def seri(image, mask):
  image = tf.io.serialize_tensor(image)
  mask = tf.io.serialize_tensor(mask)
  feature = {
      "image": _bytes_feature(image),
      "mask": _bytes_feature(mask)
  }
  return tf.train.Example(features = tf.train.Features(feature = feature)).SerializeToString()


def file_iterate_train(root, idx):
  return f"{root}AT_record_train_{idx}"

def file_iterate_test(root, idx):
  return f"{root}AT_record_test_{idx}"

def create_name_dataset(n, root, file_iterate):
## data to read

  # reading with string dataset
  filenames = []

  for idx in range(n):
    filename = file_iterate(root, idx)
    filenames.append(filename)
  return tf.data.TFRecordDataset(filenames)

def create_records(images, masks, file_iterate, root, start_index = 0):
  n = len(images)
  for idx in range(n):
    with tf.io.TFRecordWriter(file_iterate(root, idx)) as writer:
      example = seri(images[idx], masks[idx])
      writer.write(example)


## Read using the same scheme and map name dataset to value dataset

In [3]:
# prompt: read tf record

def read(record):
  features = {
      "image": tf.io.FixedLenFeature([], tf.string),
      "mask": tf.io.FixedLenFeature([], tf.string),
  }
  example = tf.io.parse_single_example(record, features)
  image = tf.io.parse_tensor(example["image"], out_type=tf.float32)
  mask = tf.io.parse_tensor(example["mask"], out_type=tf.int32)
  return image, mask

  return image, mask
def set_shape(x, y):
  x.set_shape((512, 512, 4))
  y.set_shape((512, 512, 4))
  return x, y


# test with some records

In [4]:
images = tf.random.normal((5, 512, 512, 4))
masks = tf.random.uniform((5, 512, 512, 4), minval=0, maxval = 1, dtype = tf.int32)
create_records(images, masks, root = "", file_iterate = file_iterate_train)

In [5]:
raw_dataset = create_name_dataset(n = 5, root = "", file_iterate = file_iterate_train)
batch_train = (
  raw_dataset
  # .repeat()
  .map(read)
  .map(set_shape)
  # .batch(4)
)
batch_train

<_MapDataset element_spec=(TensorSpec(shape=(512, 512, 4), dtype=tf.float32, name=None), TensorSpec(shape=(512, 512, 4), dtype=tf.int32, name=None))>

In [6]:
for x, y in batch_train.take(1):
  print(x)

tf.Tensor(
[[[-0.73771554  0.5428797  -0.21266174 -0.42254153]
  [-0.878588   -0.96039915 -0.8477328  -1.5417483 ]
  [-2.0389438  -0.77503616  0.66451156 -0.13814494]
  ...
  [ 0.50613785  2.2518826   0.38733155  1.4691987 ]
  [-0.7893158   0.52150303  1.3578594  -0.7400726 ]
  [-0.18197528 -1.0738393   0.6435553   1.2284352 ]]

 [[ 1.1061462   0.43441355  0.54651463  0.20056757]
  [-0.28315744 -0.93721884  1.0905102  -1.3710856 ]
  [-0.75930136  0.09591439 -1.7745305   0.73714745]
  ...
  [-0.522002    1.2899065   0.28151911 -1.5669826 ]
  [-0.6341211   0.48755044  0.6567216  -0.62318075]
  [ 0.22375044 -0.44927344 -0.20475571 -0.6817778 ]]

 [[ 0.49508092  0.2607163   1.5350058   0.26561326]
  [ 1.5483361  -0.1915519  -0.65738124 -0.04399431]
  [ 0.26464003 -0.23597832 -1.0910879  -2.421074  ]
  ...
  [ 0.8945367   0.6507948  -1.0306449   0.22622263]
  [ 1.0986787  -0.39106786  1.1618446   2.816444  ]
  [-0.49494603 -1.7252967   0.7089955  -0.60223675]]

 ...

 [[ 1.5605109  -0.23037